In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.optimize import minimize
from ula import GaussianMixtureULA
from Metropolized_Transition_Operator import MetropolizedTransitionOperator
from Exploration_Component import exploration_component
from BDLS import BDLS_algo
from BDEC import BDEC

In [2]:
%load_ext autoreload
%autoreload 2

# Experiment
We plan to reproduce the experiment made in section 4.1 of the paper, i.e. target distribution is a simple
2D Gaussian Mixture.

In [ ]:
mu_0=np.array([[0,8]])
sigma_0=np.array([[0.3,0],[0,0.01]])
N_init= 2000
temp_hot=0.05

weights = np.full(4, 1/4)

means = np.array([
    [ 0.0, 8.0],
    [ 0.0, 2.0],
    [-3.0, 5.0],
    [ 3.0, 5.0],
])

covs = np.array([
    [[1.2, 0.0], [0.0, 0.01]],
    [[1.2, 0.0], [0.0, 0.01]],
    [[0.01, 0.0], [0.0, 2.0]],
    [[0.01, 0.0], [0.0, 2.0]],
])

init_particles_target=multivariate_normal.rvs(mean=mu_0[0], cov=sigma_0, 
                                              size=N_init)
init_particles_hot=multivariate_normal.rvs(mean=mu_0[0], cov=sigma_0/temp_hot, 
                                           size=N_init)
delta_t=0.005
J=50 # Time for one iteration seems very long, consider setting it to 10
T=6 
B=12
thr=2



In [68]:
bdec=BDEC(mu_0,sigma_0,1,thr,B,N_init,weights,means,covs,delta_t,T,
          init_particles_hot,init_particles_target,J,temp_hot=0.05,temp_target=1
          )

bdec.run()

0
1
2
3
4
5
0
0
1
2
3
4
5


KeyboardInterrupt: 